In [119]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import csv
import datetime
import numpy as np

# 
host = "hotel-cloud-db-dev.cy9have47g8u.eu-west-2.rds.amazonaws.com"
database = "hotelcloud"
user = "hotelcloudadmin"
password = "aX2X1i7z4CUUQihoSAdasd"
port = "5432"  # default PostgreSQL port

In [120]:
# Test Connection to HotelCloud RDS 
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Example: Execute a simple query
    cursor.execute("SELECT version();")
    
    # Fetch the result
    db_version = cursor.fetchone()
    print(f"PostgreSQL database version: {db_version}")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

finally:
    # Close the cursor and connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL database version: ('PostgreSQL 16.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)
PostgreSQL connection is closed


In [121]:
#Check Booking channel names and filter for all channels that include booking.com
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Example: Execute a simple query
    cursor.execute("SELECT DISTINCT booking_channel_name FROM booking WHERE booking_channel_name ILIKE '%booking.com%' ORDER BY booking_channel_name;")
    booking_channel_names = cursor.fetchall()

    print(booking_channel_names)
    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

finally:
    # Close the cursor and connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

[('booking.com',), ('Booking.com',), ('Booking.Com',), ('Booking.Com ',), ('BOOKING.COM',), ('BOOKING.COM ',), ('booking.com bv',), ('Booking.com B.V.',), ('BOOKING.COM BV (EUR NL)',), ('Booking.com Limited',), ('BOOKING.COM-NOCOMMINBESTCHEQUE',), ('Booking.com (Old)',), ('Booking.comVCC',), ('Booking.com VCC',), ('booking.com (Virtual card',), ('Import/Booking.com',), ('Worldwide Booking.com - Guestlink',)]
PostgreSQL connection is closed


In [122]:
#Check all rate codes for all booking.com channels
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Example: Execute a simple query
    cursor.execute("SELECT DISTINCT rate_plan_code FROM booking ORDER BY rate_plan_code;")
    rate_codes = cursor.fetchall()

    print(rate_codes)
    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

finally:
    # Close the cursor and connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

[('',), (']',), ('000',), ('0000',), ('07092024',), ('100_XMAS',), ('101_UPGRADE',), ('109_BBB',), ('109_DINE',), ('10K_FULL_21',), ('10OFF',), ('10% off Advance Purchase Rate',), ('10% off Advance Purchase Rate Room Only',), ('10% off Best Available Rate',), ('10% off Best Available Rate Room Only',), ('10_RACK_BB',), ("111-Ria's Business rate",), ('111-Rias leisure',), ('119_DBB',), ('120_STAY_FRE',), ('12995_24HR',), ('12DDR',), ('139_DBB',), ('139_SD',), ('£140.00 Corp',), ('145 + VAT INC EB',), ('149BB',), ('149_DBB',), ('£150.00 Corp',), ('155BBIN',), ('159_DBB',), ('159_SD',), ('15_BB',), ('15COVIDPACKA',), ('15OFF',), ('15% off Advance Purchase Rate',), ('15% off Advance Purchase Rate Room Only',), ('15% off Best Available Rate Room Only',), ('15_RO',), ('£160.00 Corp',), ('169DATENIGHT',), ('18APRFUNC',), ('195BB',), ('1995PP',), ('19MTEU',), ('1. BB (db/tb) 2010',), ('1. BB (db/tb) 2011',), ('1. BB (s/o) 2010',), ('1. BB (s/o) 2011',), ('1GOLF_20',), ('1N2RBBMF',), ('1N2RBBMF

In [123]:
#Read in data from all booking.com channels with rate code FLRA1

SQL_QUERY = """
WITH rate_updates AS (
    SELECT DISTINCT ON (u.hotel_id, u.date_update::date)
        u.hotel_id,
        u.rate_update_id,
        u.date_update::date AS report_date
    FROM rate_update u
    WHERE
        u.hotel_id = 6
        AND u.date_update::date >= '2022-05-01'
        AND u.date_update::date < current_date
    ORDER BY u.hotel_id, u.date_update::date, u.date_update DESC
),

-- Subquery 2: ota_rooms
-- This subquery gathers information about rooms available on Online Travel Agencies (OTAs)
ota_rooms AS (
    SELECT DISTINCT
        o.ota_room_id,
        r."name",
        o.room_id
    FROM ota_room o
    JOIN room r ON r.room_id = o.room_id
    LEFT OUTER JOIN room_category rc ON rc.room_category_id = r.room_category_id
    WHERE o.hotel_id = 6
),

-- Subquery 3: rate_data
-- This subquery collects rate information for rooms, including refundable and non-refundable rates
rate_data AS (
    SELECT
        MIN(CASE WHEN r.refundable THEN r.amount END) AS refundable_rate,
        MIN(CASE WHEN NOT r.refundable THEN r.amount END) AS non_refundable_rate,
        u.hotel_id,
        u.report_date,
        r.stay_date,
        o.name AS room_name,
        r.adultcount
    FROM
        rate_new r
    JOIN
        rate_updates u ON r.rate_update_id = u.rate_update_id
    JOIN
        ota_rooms o ON o.ota_room_id = r.ota_room_id
    GROUP BY
        u.hotel_id,
        u.report_date,
        r.stay_date,
        o.name,
        r.adultcount
),

-- Subquery 4: booking
-- This subquery gathers detailed booking information, including guest details, room information, and financial data
booking AS (
    SELECT
        p.first_name,
        p.last_name,
        b.hotel_id,
        b.room_id,
        b.created_date::date AS created_date,
        b.check_in,
        b.check_out, 
        b.cancel_date::date AS cancel_date,
        b.booking_reference, 
        EXTRACT(DAY FROM (dt."date" - b.created_date::date)) AS lead_in,
        b.booking_channel_name,
        b.booking_status, 
        b.adults,
        b.rate_plan_code,
        b.nights,
        b.room_revenue,
        ROUND(COALESCE(br.total_revenue, b.total_revenue / NULLIF(b.nights, 0)),2) AS total_revenue_x,
        Round(b.total_revenue * 1.2) as exp_rate,
        b.total_revenue_after_tax,
        h."name" AS hotel_name,
        r."name" AS room_name,
        r.code as room_code,
        dt."date" AS stay_date
    FROM
        booking b
    JOIN profile p ON b.profile_id = p.profile_id
    JOIN hotel h ON b.hotel_id = h.hotel_id
    JOIN room r ON b.room_id = r.room_id
    JOIN caldate dt ON b.check_in <= dt."date"
        AND (b.check_out > dt."date" OR (b.check_in = b.check_out AND dt."date" = b.check_in))
    LEFT JOIN booking_rate br ON b.booking_id = br.booking_rate_id
    WHERE
        b.created_date >= '2022-01-01'
        AND b.hotel_id = '6'
        AND b.booking_channel_name IN (
 'booking.com', 'Booking.com', 'Booking.Com', 'Booking.Com ', 'BOOKING.COM',
 'BOOKING.COM ', 'booking.com bv', 'Booking.com B.V.', 'BOOKING.COM BV (EUR NL)',
 'Booking.com Limited', 'BOOKING.COM-NOCOMMINBESTCHEQUE', 'Booking.com (Old)',
 'Booking.comVCC', 'Booking.com VCC', 'booking.com (Virtual card',
 'Import/Booking.com', 'Worldwide Booking.com - Guestlink'
)
        AND b.nights = 1
        AND b.rate_plan_code = 'FLRA1'
)

-- Main Query
-- This part combines the booking data with the rate data and filters for discrepancies between expected and actual rates
SELECT b.*, r.*
FROM booking AS b
JOIN caldate dt ON b.check_in <= dt."date"
    AND (b.check_out > dt."date" OR (b.check_in = b.check_out AND dt."date" = b.check_in))
LEFT JOIN rate_data AS r ON r.adultcount = b.adults 
    AND r.room_name = b.room_name 
    AND b.created_date = r.report_date 
    AND dt.date = r.stay_date
WHERE b.exp_rate <> r.refundable_rate
ORDER BY b.nights ASC;
"""


try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Example: Execute a simple query
    cursor.execute(SQL_QUERY)
    results = cursor.fetchall()

    column_names = [desc[0] for desc in cursor.description]

    filename = f"full_rates_data.csv"

    # Write to CSV
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write header
        csvwriter.writerow(column_names)
        
        # Write data rows
        csvwriter.writerows(results)

    print(f"Data has been written to {filename}")

    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

finally:
    # Close the cursor and connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Data has been written to full_rates_data.csv
PostgreSQL connection is closed


In [131]:
full_rates_data = pd.read_csv('full_rates_data.csv')
# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(f"Number of rows: {len(full_rates_data)}")

full_rates_data.head()

Number of rows: 1477


,first_name,last_name,hotel_id,room_id,created_date,check_in,check_out,cancel_date,booking_reference,lead_in,booking_channel_name,booking_status,adults,rate_plan_code,nights,room_revenue,total_revenue_x,exp_rate,total_revenue_after_tax,hotel_name,room_name,room_code,stay_date,refundable_rate,non_refundable_rate,hotel_id.1,report_date,stay_date.1,room_name.1,adultcount
0,Aidan,Darcy,6,127,2022-06-29,2022-10-01,2022-10-02,NaN,376297932,94,BOOKING.COM BV (EUR NL),CheckedOut,2,FLRA1,1,232.12,232.12,279,278.54,Mercure Hyde Park,Standard King,KGC,2022-10-01 00:00:00,309.0,NaN,6,2022-06-29,2022-10-01,Standard King,2
1,Maciej,Madejak,6,125,2022-05-07,2022-05-09,2022-05-10,2022-05-09,375274226,2,BOOKING.COM BV (EUR NL),Cancelled,1,FLRA1,1,0.00,0.00,0,0.00,Mercure Hyde Park,Family Room,TRB,2022-05-09 00:00:00,369.0,NaN,6,2022-05-07,2022-05-09,Family Room,1
2,Sophie,Wadden,6,127,2022-08-01,2022-09-18,2022-09-19,NaN,376923042,48,BOOKING.COM BV (EUR NL),CheckedOut,2,FLRA1,1,172.12,172.12,207,206.54,Mercure Hyde Park,Standard King,KGC,2022-09-18 00:00:00,229.0,209.0,6,2022-08-01,2022-09-18,Standard King,2
3,Lydia,Johnson,6,127,2022-08-01,2022-08-12,2022-08-13,NaN,376915764,11,BOOKING.COM BV (EUR NL),CheckedOut,2,FLRA1,1,209.62,209.62,252,251.54,Mercure Hyde Park,Standard King,KGC,2022-08-12 00:00:00,279.0,249.0,6,2022-08-01,2022-08-12,Standard King,2
4,Sharifah,Iyliana,6,127,2022-08-01,2022-08-21,2022-08-22,NaN,377289468,20,BOOKING.COM BV (EUR NL),CheckedOut,2,FLRA1,1,142.12,142.12,171,170.54,Mercure Hyde Park,Standard King,KGC,2022-08-21 00:00:00,199.0,189.0,6,2022-08-01,2022-08-21,Standard King,2


In [132]:
rates_comparison_df = full_rates_data[['booking_reference', 'room_revenue', 'total_revenue_after_tax', 'refundable_rate']]
rates_comparison_df.head()

,booking_reference,room_revenue,total_revenue_after_tax,refundable_rate
0,376297932,232.12,278.54,309.0
1,375274226,0.00,0.00,369.0
2,376923042,172.12,206.54,229.0
3,376915764,209.62,251.54,279.0
4,377289468,142.12,170.54,199.0


In [136]:
#Calculate total revenue after taxes and discount
#Tax + Discount = 10% off (Genius Level 1)
rates_comparison_df['total_revenue_after_tax_and_discount'] = rates_comparison_df['room_revenue']*1.2/0.9
#Tax + 2 Discounts => 10% off (Genius Level 1) + 10% off (App Discount)
rates_comparison_df['total_revenue_after_tax_and_two_discounts'] = rates_comparison_df['room_revenue']*1.2/0.81
#Tax + Discount = 15% off (Genius Level 2)
rates_comparison_df['total_revenue_after_tax_and_discount_g2'] = rates_comparison_df['room_revenue']*1.2/0.85
rates_comparison_df['total_revenue_after_tax_and_discount_g2_and_app_discount'] = rates_comparison_df['room_revenue']*1.2/0.765
#Tax + Discount = 20% off (Genius Level 3)
rates_comparison_df['total_revenue_after_tax_and_discount_g3'] = rates_comparison_df['room_revenue']*1.2/0.80
rates_comparison_df['total_revenue_after_tax_and_discount_g3_and_app_discount'] = rates_comparison_df['room_revenue']*1.2/0.72

/var/folders/7m/8y6jbsj56511tmcdmrpyjlrr0000gn/T/ipykernel_66832/2421277455.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rates_comparison_df['total_revenue_after_tax_and_discount'] = rates_comparison_df['room_revenue']*1.2/0.9
/var/folders/7m/8y6jbsj56511tmcdmrpyjlrr0000gn/T/ipykernel_66832/2421277455.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rates_comparison_df['total_revenue_after_tax_and_two_discounts'] = rates_comparison_df['room_revenue']*1.2/0.81
/var/folders/7m/8y6jbsj56511tmcdmrpyjl

In [137]:
rates_comparison_df.head()

,booking_reference,room_revenue,total_revenue_after_tax,refundable_rate,total_revenue_after_tax_and_discount,total_revenue_after_tax_and_two_discounts,total_revenue_after_tax_and_discount_g2,total_revenue_after_tax_and_discount_g2_and_app_discount,total_revenue_after_tax_and_discount_g3,total_revenue_after_tax_and_discount_g3_and_app_discount
0,376297932,232.12,278.54,309.0,309.493333,343.881481,327.698824,364.109804,348.18,386.866667
1,375274226,0.00,0.00,369.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,376923042,172.12,206.54,229.0,229.493333,254.992593,242.992941,269.992157,258.18,286.866667
3,376915764,209.62,251.54,279.0,279.493333,310.548148,295.934118,328.815686,314.43,349.366667
4,377289468,142.12,170.54,199.0,189.493333,210.548148,200.640000,222.933333,213.18,236.866667


In [139]:
# Check for matches and mismatches between total revenue after taxes and discount and refundable rate
match_after_tax_and_discount = np.isclose(rates_comparison_df['total_revenue_after_tax_and_discount'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_discount = match_after_tax_and_discount.sum()
num_mismatches_tax_discount = (~match_after_tax_and_discount).sum()

print(f"Comparison of total revenue after tax and Genius Level 1 discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_discount}, Percentage of matches: {num_matches_tax_discount/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_discount}")

# Check for matches and mismatches between total revenue after taxes and two discounts and refundable rate
match_after_tax_and_two_discounts = np.isclose(rates_comparison_df['total_revenue_after_tax_and_two_discounts'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_two_discounts = match_after_tax_and_two_discounts.sum()
num_mismatches_tax_two_discounts = (~match_after_tax_and_two_discounts).sum()

print(f"\nComparison of total revenue after tax and Genius Level 1 discount and App Discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_two_discounts}, Percentage of matches: {num_matches_tax_two_discounts/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_two_discounts}")

# Check for matches and mismatches between total revenue after taxes and discount and refundable rate
match_after_tax_and_discount_g2 = np.isclose(rates_comparison_df['total_revenue_after_tax_and_discount_g2'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_discount_g2 = match_after_tax_and_discount_g2.sum()
num_mismatches_tax_discount_g2 = (~match_after_tax_and_discount_g2).sum()

print(f"\nComparison of total revenue after tax and Genius Level 2 discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_discount_g2}, Percentage of matches: {num_matches_tax_discount_g2/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_discount_g2}")

match_after_tax_and_discount_g2_and_app_discount = np.isclose(rates_comparison_df['total_revenue_after_tax_and_discount_g2_and_app_discount'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_discount_g2_and_app_discount = match_after_tax_and_discount_g2_and_app_discount.sum()
num_mismatches_tax_discount_g2_and_app_discount = (~match_after_tax_and_discount_g2_and_app_discount).sum()

print(f"\nComparison of total revenue after tax and Genius Level 2 discount and App Discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_discount_g2_and_app_discount}, Percentage of matches: {num_matches_tax_discount_g2_and_app_discount/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_discount_g2_and_app_discount}")


match_after_tax_and_discount_g3 = np.isclose(rates_comparison_df['total_revenue_after_tax_and_discount_g3'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_discount_g3 = match_after_tax_and_discount_g3.sum()
num_mismatches_tax_discount_g3 = (~match_after_tax_and_discount_g3).sum()

print(f"\nComparison of total revenue after tax and Genius Level 3 discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_discount_g3}, Percentage of matches: {num_matches_tax_discount_g3/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_discount_g3}")

match_after_tax_and_discount_g3_and_app_discount = np.isclose(rates_comparison_df['total_revenue_after_tax_and_discount_g3_and_app_discount'], rates_comparison_df['refundable_rate'], atol=1)
num_matches_tax_discount_g3_and_app_discount = match_after_tax_and_discount_g3_and_app_discount.sum()
num_mismatches_tax_discount_g3_and_app_discount = (~match_after_tax_and_discount_g3_and_app_discount).sum()

print(f"\nComparison of total revenue after tax and Genius Level 3 discount and App Discount with refundable rate:")
print(f"Number of matches: {num_matches_tax_discount_g3_and_app_discount}, Percentage of matches: {num_matches_tax_discount_g3_and_app_discount/len(rates_comparison_df)*100}")
print(f"Number of mismatches: {num_mismatches_tax_discount_g3_and_app_discount}, Percentage of mismatches: {num_mismatches_tax_discount_g3_and_app_discount/len(rates_comparison_df)*100}")






Comparison of total revenue after tax and Genius Level 1 discount with refundable rate:
Number of matches: 93, Percentage of matches: 6.296547054840894
Number of mismatches: 1384

Comparison of total revenue after tax and Genius Level 1 discount and App Discount with refundable rate:
Number of matches: 79, Percentage of matches: 5.348679756262695
Number of mismatches: 1398

Comparison of total revenue after tax and Genius Level 2 discount with refundable rate:
Number of matches: 18, Percentage of matches: 1.2186865267433988
Number of mismatches: 1459

Comparison of total revenue after tax and Genius Level 2 discount and App Discount with refundable rate:
Number of matches: 35, Percentage of matches: 2.3696682464454977
Number of mismatches: 1442

Comparison of total revenue after tax and Genius Level 3 discount with refundable rate:
Number of matches: 18, Percentage of matches: 1.2186865267433988
Number of mismatches: 1459

Comparison of total revenue after tax and Genius Level 3 discou

In [140]:
# Create boolean masks for matches
match_refundable_vs_tax_discount = np.isclose(rates_comparison_df['refundable_rate'], 
                                              rates_comparison_df['total_revenue_after_tax_and_discount'], 
                                              atol=1)

match_refundable_vs_tax_two_discounts = np.isclose(rates_comparison_df['refundable_rate'], 
                                                   rates_comparison_df['total_revenue_after_tax_and_two_discounts'], 
                                                   atol=1)

match_refundable_vs_tax_discount_g2 = np.isclose(rates_comparison_df['refundable_rate'], 
                                                   rates_comparison_df['total_revenue_after_tax_and_discount_g2'], 
                                                   atol=1)

match_refundable_vs_tax_discount_g2_and_app_discount = np.isclose(rates_comparison_df['refundable_rate'], 
                                                   rates_comparison_df['total_revenue_after_tax_and_discount_g2_and_app_discount'], 
                                                   atol=1)  

match_refundable_vs_tax_discount_g3 = np.isclose(rates_comparison_df['refundable_rate'], 
                                                   rates_comparison_df['total_revenue_after_tax_and_discount_g3'], 
                                                   atol=1)

match_refundable_vs_tax_discount_g3_and_app_discount = np.isclose(rates_comparison_df['refundable_rate'], 
                                                   rates_comparison_df['total_revenue_after_tax_and_discount_g3_and_app_discount'], 
                                                   atol=1)




# Create a mask for all mismatches
all_mismatches = (~match_refundable_vs_tax_discount) & (~match_refundable_vs_tax_two_discounts) & (~match_refundable_vs_tax_discount_g2) & (~match_refundable_vs_tax_discount_g2_and_app_discount) & (~match_refundable_vs_tax_discount_g3) & (~match_refundable_vs_tax_discount_g3_and_app_discount)

# Create a DataFrame with all mismatches
mismatches_df = rates_comparison_df[all_mismatches]

# Print summary
print(f"Total number of rows: {len(rates_comparison_df)}")
print(f"Number of mismatches (not matching single or double discounts): {len(mismatches_df)}")

# Round numeric columns to the nearest integer
numeric_columns = mismatches_df.select_dtypes(include=[np.number]).columns
mismatches_df_rounded = mismatches_df.copy()
mismatches_df_rounded[numeric_columns] = mismatches_df_rounded[numeric_columns].round().astype(int)

# Display the first few rows of the mismatches DataFrame with rounded values
print("\nFirst few rows of all mismatches (values rounded to nearest integer):")
mismatches_df_rounded.head()


Total number of rows: 1477
Number of mismatches (not matching single or double discounts): 1220

First few rows of all mismatches (values rounded to nearest integer):


,booking_reference,room_revenue,total_revenue_after_tax,refundable_rate,total_revenue_after_tax_and_discount,total_revenue_after_tax_and_two_discounts,total_revenue_after_tax_and_discount_g2,total_revenue_after_tax_and_discount_g2_and_app_discount,total_revenue_after_tax_and_discount_g3,total_revenue_after_tax_and_discount_g3_and_app_discount
1,375274226,0,0,369,0,0,0,0,0,0
4,377289468,142,171,199,189,211,201,223,213,237
5,376918586,142,171,199,189,211,201,223,213,237
6,376918600,183,219,279,244,271,258,287,274,305
7,379446051,128,153,199,171,189,181,201,192,213


In [141]:
def ends_with_9(x):
    return np.abs(x % 10 - 9) < 0.1 # Using a small threshold to account for floating-point precision
# Create boolean masks for each condition
ends_9_after_tax = ends_with_9(rates_comparison_df['total_revenue_after_tax'])
ends_9_after_tax_discount = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_discount'])
ends_9_after_tax_two_discounts = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_two_discounts'])
ends_9_after_tax_discount_g2 = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_discount_g2'])
ends_9_after_tax_discount_g2_and_app_discount = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_discount_g2_and_app_discount'])
ends_9_after_tax_discount_g3 = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_discount_g3'])
ends_9_after_tax_discount_g3_and_app_discount = ends_with_9(rates_comparison_df['total_revenue_after_tax_and_discount_g3_and_app_discount'])
# Combine the masks
any_ends_with_9 = ends_9_after_tax | ends_9_after_tax_discount | ends_9_after_tax_two_discounts | ends_9_after_tax_discount_g2 | ends_9_after_tax_discount_g2_and_app_discount | ends_9_after_tax_discount_g3 | ends_9_after_tax_discount_g3_and_app_discount
# Create the new DataFrame for possible upgrades
possible_upgrades_df = rates_comparison_df[any_ends_with_9]
# Print summary
print(f"Number of possible upgrades (i.e. ending with 9): {len(possible_upgrades_df)}")
# Round the displayed values to 2 decimal places for clarity
upgrades_display = possible_upgrades_df[['booking_reference', 'total_revenue_after_tax',
'total_revenue_after_tax_and_discount',
'total_revenue_after_tax_and_two_discounts']].round()
print("\nFirst few rows of possible upgrades (rounded to 2 decimal places):")
upgrades_display.head()

Number of possible upgrades (i.e. ending with 9): 24

First few rows of possible upgrades (rounded to 2 decimal places):


,booking_reference,total_revenue_after_tax,total_revenue_after_tax_and_discount,total_revenue_after_tax_and_two_discounts
14,595939158056,791.0,879.0,977.0
55,380175627,307.0,341.0,379.0
155,375522882,404.0,449.0,499.0
231,595939028806,341.0,379.0,421.0
234,595939565149,307.0,341.0,379.0
